In [1]:
# !/usr/bin/python3 -m pip install ipykernel -U --user --force-reinstall

In [2]:
# !pip3 install pyvis
!pip3 install bokeh

In [3]:
import pandas as pd
from tqdm import tqdm
import numpy as np
import re
import html
from urllib import parse
import requests
import networkx
from bokeh.io import output_notebook, show, save
output_notebook()
# from pyvis.network import Network

Loading BokehJS ...

In [4]:
from bokeh.io import output_notebook, show, save
from bokeh.models import Range1d, Circle, ColumnDataSource, MultiLine
from bokeh.plotting import figure
from bokeh.plotting import from_networkx

In [5]:
from utils import *
max_node = 50

In [6]:
with open('./soc-pokec-profiles.txt', 'r') as f:
    profiles = f.readlines()
with open('./soc-pokec-relationships.txt', 'r') as f:
    relationships = f.readlines()

relationships = relationships
profiles = profiles[:max_node]
print(len(profiles))

with open('profiles_50_sample.txt', 'w') as f:
    for l in profiles:
        f.write(l)

# with open('relationships_50_sample.txt', 'w') as f:
#     for l in relationships:
#         f.write(l)

with open('profiles_50_sample.txt', 'r') as f:
    profiles = f.readlines()
print(len(profiles))
# with open('relationships_sample.txt', 'r') as f:
#     relationships = f.readlines()

50
50


In [7]:
node_attributes = ['user_id', 'public', 'completion_percentage', 'gender', 'region', 'last_login', 'registration', 'AGE', 'body', 'I_am_working_in_field', 'spoken_languages', 'hobbies', 'I_most_enjoy_good_food', 'pets', 'body_type', 'my_eyesight', 'eye_color', 'hair_color', 'hair_type', 'completed_level_of_education', 'favourite_color', 'relation_to_smoking', 'relation_to_alcohol', 'sign_in_zodiac', 'on_pokec_i_am_looking_for', 'love_is_for_me', 'relation_to_casual_sex', 'my_partner_should_be', 'marital_status', 'children', 'relation_to_children', 'I_like_movies', 'I_like_watching_movie', 'I_like_music', 'I_mostly_like_listening_to_music', 'the_idea_of_good_evening', 'I_like_specialties_from_kitchen', 'fun', 'I_am_going_to_concerts', 'my_active_sports', 'my_passive_sports', 'profession', 'I_like_books', 'life_style', 'music', 'cars', 'politics', 'relationships', 'art_culture', 'hobbies_interests', 'science_technologies', 'computers_internet', 'education', 'sport', 'movies', 'travelling', 'health', 'companies_brands', 'more']

In [8]:
parsed_profiles = []

for p in tqdm(profiles):
    parsed_profiles.append(parse_row(p))

from deep_translator import (GoogleTranslator,
                             MicrosoftTranslator,
                             PonsTranslator,
                             LingueeTranslator,
                             MyMemoryTranslator,
                             YandexTranslator,
                             PapagoTranslator,
                             DeeplTranslator,
                             QcriTranslator,
                             single_detection,
                             batch_detection)

translator = GoogleTranslator(source='sk', target='en')


def translate(translator, text):
    if text == 'null':
        return text
    elif text.isnumeric():
        return int(text)
    else:
        try:
            t =  translator.translate(text)
        except:
            t = text
        return t

data = {}
for i, a in tqdm(enumerate(node_attributes)):
    data[a] = [translate(translator, p[i]) for p in parsed_profiles]

df = pd.DataFrame(data)
df.to_csv('toy_example.csv', index=False)

100%|██████████| 50/50 [00:00<00:00, 145838.11it/s]
59it [03:09,  3.21s/it]


In [9]:
def load_from_csv(path):
    df = pd.read_csv(path)
    return df

def row_to_dict(row, attr):
    data = {}
    for a in attr:
        data[a] = row[a]
    return data

In [10]:
# for i in range(len(node_attributes)):
#     a = node_attributes[i]
#     try:
#         print('column name', a, 'count null', df[a].value_counts()['null'])
#     except:
#         print('column name', a, 'count null', 0)

column name user_id count null 0
column name public count null 0
column name completion_percentage count null 0
column name gender count null 0
column name region count null 0
column name last_login count null 0
column name registration count null 0
column name AGE count null 0
column name body count null 19
column name I_am_working_in_field count null 25
column name spoken_languages count null 14
column name hobbies count null 15
column name I_most_enjoy_good_food count null 20
column name pets count null 17
column name body_type count null 22
column name my_eyesight count null 21
column name eye_color count null 11
column name hair_color count null 15
column name hair_type count null 21
column name completed_level_of_education count null 18
column name favourite_color count null 12
column name relation_to_smoking count null 19
column name relation_to_alcohol count null 19
column name sign_in_zodiac count null 14
column name on_pokec_i_am_looking_for count null 17
column name love_is_

In [11]:
def create_network(profiles, relationships, attr, max_node):
    
    net = get_graph_edge(relationships, max_node)
    # profiles : dataframe

    all_attrs = {}
    for idx, p in profiles.iterrows():
        node_attr = row_to_dict(p, attr)
        node_id = int(p['user_id'])
        all_attrs[node_id] = node_attr
    networkx.set_node_attributes(net, all_attrs)
    return net

In [12]:
net = create_network(df, relationships, node_attributes, max_node)

Edge Amount of this graph: 127


In [13]:
net.nodes[3]

{'user_id': 3,
 'public': 0,
 'completion_percentage': 38,
 'gender': 1,
 'region': 'Bratislava Region, Bratislava - Karlova Village',
 'last_login': '2012-05-10 18:05:00.0',
 'registration': '2010-05-23 00:00:00.0',
 'AGE': 29,
 'body': 'null',
 'I_am_working_in_field': 'advertising and media, services and trade',
 'spoken_languages': 'English, German',
 'hobbies': 'sports, travel',
 'I_most_enjoy_good_food': 'null',
 'pets': 'null',
 'body_type': 'null',
 'my_eyesight': 'null',
 'eye_color': 'green',
 'hair_color': 'brown',
 'hair_type': 'null',
 'completed_level_of_education': 'null',
 'favourite_color': 'black, blue',
 'relation_to_smoking': 'null',
 'relation_to_alcohol': 'null',
 'sign_in_zodiac': 'null',
 'on_pokec_i_am_looking_for': 'a good friend, girlfriend, someone to chat with',
 'love_is_for_me': 'null',
 'relation_to_casual_sex': 'null',
 'my_partner_should_be': 'null',
 'marital_status': 'single',
 'children': 'null',
 'relation_to_children': 'null',
 'I_like_movies': 'a

In [15]:
title = 'Testing Network'

HOVER_TOOLTIPS = [(p, "@"+p) for p in node_attributes]
plot = figure(tooltips = HOVER_TOOLTIPS,
              tools="pan,wheel_zoom,save,reset", active_scroll='wheel_zoom',
            x_range=Range1d(-10.1, 10.1), y_range=Range1d(-10.1, 10.1), title=title)

network_graph = from_networkx(net, networkx.spring_layout, scale=10, center=(0, 0))

#Set node size and color
network_graph.node_renderer.glyph = Circle(size=15, fill_color='skyblue')

#Set edge opacity and width
network_graph.edge_renderer.glyph = MultiLine(line_alpha=0.5, line_width=1)
plot.renderers.append(network_graph)
show(plot)